In [1]:
import pandas as pd
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
from sklearn import  linear_model
from sklearn.preprocessing import  PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [2]:
def readDeathData(country):
    """
    Liest Daten ueber die Anzahl der Tote aus
    :param country: Land
    :return: ausgelesene Daten
    """
    death_data = pd.read_csv('csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
    data = death_data[(death_data['Country/Region'] == country)].iloc[:,4:]
    return data.sum()

def readConfirmedData(country):
    """
    liest die Daten ueber die Anzahl der Infizierten ein
    :param country: Land
    :return: ausgelesene Daten
    """
    recover_data = pd.read_csv('csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    data = recover_data[(recover_data['Country/Region'] == country)].iloc[:,4:]
    return data.sum()

def readRecoveredData(country):
    """
    liest die Daten ueber die Anzahl der Genesenen ein
    :param country: Land
    :return: ausgelesene Daten
    """
    recover_data = pd.read_csv('csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
    data = recover_data[(recover_data['Country/Region'] == country)].iloc[:,4:]
    return data.sum()

In [3]:
#Populationsstand verschiedener Laender
popu_dict = {'Germany':82927922, 'Italy':60482200}

def load_IR(name, N, t,T=None):
    """
    Laedt die Daten der Faelle, Genesenen und Toten
    :param name: Das zu betrachtende Land
    :param N: Gesamtpopulation
    :param t: Startzeitpunkt
    :param T: Endzeitpunkt (optional)
    :return: Aktive Infizierte, Genesen, Tote, gesamte Faelle
    """
    #T=163

    R = readRecoveredData(name)
    D = readDeathData(name)
    C = readConfirmedData(name)
    R = R.tolist()[t:T]
    D = D.tolist()[t:T]
    C = C.tolist()[t:T]
    R = np.array(R)
    D = np.array(D)
    C = np.array(C)
    I = C - R - D
    return I,R,D,C

def SSE(I_t, I):
    """
    Summe des quadratischen Fehlers
    :param I_t: modellierte Infizierte
    :param I: Infizierte aus Daten
    :return: Summe des quadratischen Fehlers
    """
    sse = (I_t[:len(I)] - I)**2
    return sse.sum()

def RungeKutta(beta, gamma, N, S_0, I_0, R_0, t):
    """
    Runge-Kutta-Verfahren zum loesen des SIR-Modells
    :param beta: Kontaktrate
    :param gamma: Genseungsrate
    :param N: Gesamtpopulation
    :param S_0: Anfangswert des Anfaelligen
    :param I_0: Anfangswert des Infizierten
    :param R_0: Anfangswert der Toten/Genesenen
    :param t: Startzeitpunkt
    :return: geloestes SIR-Modell ab t mit S_t, I_t, R_t
    """
    days = 200
    X = np.arange(t, days)
    I_t = np.zeros(days)
    S_t = np.zeros(days)
    R_t = np.zeros(days)
    I_t[0] = I_0
    S_t[0] = S_0
    R_t[0] = R_0
    h = 1
    for i in range(1, days):
        k11 = -beta * S_t[i-1] * I_t[i-1] / N
        k21 = beta * S_t[i-1] * I_t[i-1] / N - gamma * I_t[i-1]
        k31 = gamma * I_t[i-1]

        k12 = -beta * (S_t[i-1] + h / 2 * k11) * (I_t[i-1] + h / 2 * k21) / N
        k22 = beta * (S_t[i-1] + h / 2 * k11) * (I_t[i-1] + h / 2 * k21) / N - gamma * (I_t[i-1] + h / 2 * k21)
        k32 = gamma * (I_t[i-1] + h / 2 * k21)

        k13 = -beta * (S_t[i-1] + h / 2 * k12) * (I_t[i-1] + h / 2 * k22) / N
        k23 = beta * (S_t[i-1] + h / 2 * k12) * (I_t[i-1] + h / 2 * k22) / N - gamma * (I_t[i-1] + h / 2 * k22)
        k33 = gamma * (I_t[i-1] + h / 2 * k22)

        k14 = -beta * (S_t[i-1] + h * k13) * (I_t[i-1] + h * k23) / N
        k24 = beta * (S_t[i-1] + h * k13) * (I_t[i-1] + h * k23) / N - gamma * (I_t[i-1] + h * k23)
        k34 = gamma * (I_t[i-1] + h * k23)

        S_t[i] = S_t[i-1] + h / 6 * (k11 + 2 * k12 + 2 * k13 + k14)
        I_t[i] = I_t[i-1] + h / 6 * (k21 + 2 * k22 + 2 * k23 + k24)
        R_t[i] = R_t[i-1] + h / 6 * (k31 + 2 * k32 + 2 * k33 + k34)
    return S_t[t:],I_t[t:],R_t[t:]

def create_assist_date(datestart = None,dateend = None):
    """
    Erstellt eine Liste von Daten
    :param datestart: Startdatum
    :param dateend: Enddatum
    :return: Liste von Daten zwischen datestart und dateend
    """
    if datestart is None:
        datestart = '2016-01-01'
    if dateend is None:
        dateend = datetime.datetime.now().strftime('%Y-%m-%d')

    datestart=datetime.datetime.strptime(datestart,'%Y-%m-%d')
    dateend=datetime.datetime.strptime(dateend,'%Y-%m-%d')
    date_list = [datestart.strftime('%Y-%m-%d')]
    while datestart<dateend:
        datestart+=datetime.timedelta(days=+1)
        date_list.append(datestart.strftime('%Y-%m-%d'))
    return date_list

def predictPlot(I, I_t, t=None):
    """
    Plot von I und I_t
    :param I: gemessenen Infizierten
    :param I_t: modellierte oder vorhergesagt Infizierte
    :param t: Start ab wann vorhergesagt wird
    """
    #date_X = create_assist_date("2020-1-22", "2020-10-01")
    x_test  =pd.date_range(start='1/22/2020', end='10/1/2023', freq='D')
    X = np.arange(0, len(I_t))
    ax  = plt.figure(figsize=(14, 8))
    plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.minorticks_on()
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    sns.lineplot(x_test[:len(I)], I, label = 'echte Daten')
    sns.set_style('whitegrid')
    if t is not None:
        sns.lineplot(x_test[0:t],I_t[0:t],label="Modelliert")
        sns.lineplot(x_test[t:len(I_t)], I_t[t:], label='Vorhersage')
    else:
        sns.lineplot(x_test[:len(I_t)],I_t,label="Modelliert")
    plt.xlabel('Zeit')
    plt.ylabel('Anzahl aktiver Infizierten')
    plt.title('SIR Modell')

def oneDayError(I,I_p):
    """
    Vorhersage Fehler
    :param I: gemessene Daten
    :param I_p: modellierte oder vorhergesagte Daten
    :return: Liste der Fehler
    """
    error = ((I_p[:len(I)] - I)/I)
    X = np.arange(0, len(error))
    x_test  =pd.date_range(start='1/22/2020', end='10/1/2023', freq='D')
    ax  = plt.figure(figsize=(13, 8))
    sns.set_style('darkgrid')
    sns.lineplot(x_test[:len(error)],error, label="Error")
    plt.xlabel('Date')
    plt.ylabel('Size of Error')
    plt.title('Prediction Error')
    plt.savefig('error.png')
    return error

def read_I_t(I,country, size):
    """
    Liest die bereits modellierten Werte von I_t ein
    :param I: gemessene I
    :param country: Land
    :param size: Groesse des Zeitfenster
    :return: Liste der modellierten I
    """
    I_t_list = []
    I_t_all = []
    I_t_p = list(I[0:size])

    for i in range(0, len(I)- size +1 ):
        if i == len(I) - size:
            j = len(I) - 1
        else:
            j = i + size - 1
        file_name = str(i) + '_'+str(j)+'_It.csv'
        a = pd.read_csv('data/'+country+'/'+str(size)+'/'+file_name,engine='python')
        I_t = list(a['I_t'])
        I_t_list.append(I_t)

        if i == len(I) - size:
            I_t_all = I_t_all + I_t
            I_t_p = I_t_p + I_t[size:]
        else:
            I_t_all = I_t_all + I_t[:1]
            I_t_p.append(I_t[size])
    return I_t_p,I_t_all


def read_opt(I,country, size):
    """
    liest die optimalen Parameter
    :param I: gemessene I
    :param country: Land
    :param size: Groesse des Zeitfenster
    :return: Liste von optimalen beta und gamma
    """
    beta_list = []
    gamma_list = []
    for i in range(0, len(I)- size+1):
        if i == len(I) - size:
            j = len(I) - 1
        else:
            j = i + size-1
        file_name = str(i) + '_'+str(j)+'_opt.csv'
        a = pd.read_csv('data/'+country+'/'+str(size)+'/'+file_name,engine='python')
        #opt = a.iloc[a['sse'].idxmin(),][1:3].tolist()
        opt = a.iloc[0,1:3].tolist()
        beta_list.append(opt[0])
        gamma_list.append(opt[1])
    beta_list = np.array(beta_list,dtype="float64")
    gamma_list = np.array(gamma_list,dtype="float64")
    return beta_list,gamma_list

def plot(data,label=None):
    X = np.arange(0, len(data))
    ax  = plt.figure(figsize=(13, 8))
    sns.lineplot(X[:len(data)],data,label=label)


In [4]:

minMSE = float('inf')
opt = 0

def predictParam(size, param):
    datasets_X = np.arange(0, len(param)).reshape([len(param),1])
    param_p = list(param[0:size])
    for j in range(size, len(param)):
        i = j - size + 1
        nextY = paramPredict(datasets_X[i:j], param[i:j])
        param_p.append(nextY)
    param_p = np.array(param_p)
    print('mse:',mean_squared_error(param_p, param),SSE(param,param_p))

    X = np.arange(0, len(param))
    ax  = plt.figure(figsize=(13, 8))
    sns.lineplot(X,param,label="param(t)")
    sns.lineplot(X, param_p, label = 'predict')
    plt.xlabel('Date')
    plt.ylabel('number')
    return param_p

def paramPredict(datasets_X,datasets_Y):
    poly_reg =PolynomialFeatures(degree=2)
    X_ploy =poly_reg.fit_transform(datasets_X)
    lin_reg_2=linear_model.LinearRegression()
    lin_reg_2.fit(X_ploy,datasets_Y)
    y_predict = lin_reg_2.predict(poly_reg.fit_transform(datasets_X))
    nextX = np.array(datasets_X[-1][0]+1).reshape([1,1])
    nextY = lin_reg_2.predict(poly_reg.fit_transform(nextX))
    #nextY ist nur eine Zahl
    return nextY

def polyPredict(datasets_X,datasets_Y):
    poly_reg =PolynomialFeatures(degree=2)
    X_ploy =poly_reg.fit_transform(datasets_X)
    lin_reg_2=linear_model.LinearRegression()
    lin_reg_2.fit(X_ploy,datasets_Y)
    y_predict = lin_reg_2.predict(poly_reg.fit_transform(datasets_X))
    mse = mean_squared_error(datasets_Y, y_predict)
    print('mse: ', mse,SSE(datasets_Y, y_predict))
    plt.scatter(datasets_X,datasets_Y,color='red')
    plt.plot(datasets_X,y_predict,color='blue')
    plt.xlabel('Area')
    plt.ylabel('Price')
    plt.show()
    return mse

def saveParam(country, size,t):
    I,R,D,C = load_IR(country,0)
    beta_list,gamma_list = read_opt(I,country,size)
    r0,dif = parameterPlot(beta_list,gamma_list)
    where_are_NaNs = np.isnan(r0)
    r0[where_are_NaNs] = 0
    where_are_NaNs = np.isnan(dif)
    dif[where_are_NaNs] = 0
    beta_list = beta_list[t:]
    gamma_list = gamma_list[t:]
    r0 = r0[t:]
    dif = dif[t:]
    I = I[t:]
    plot(r0)
    plot(dif)
    r0_p = predictParam(size, r0)
    dif_p = predictParam(size, dif)
    print("s", len(r0), len(r0_p), len(I))
    dirs = '\\data\\'
    dir_path = os.path.dirname(os.path.abspath('__file__')) + dirs
    dir_path = dir_path+country+'\\'
    date_X = create_assist_date("2020-1-22", "2021-10-01")
    date_X = date_X[t:]
    data = np.vstack((date_X[:len(I)],beta_list,gamma_list,r0,dif,r0_p,dif_p))
    data = pd.DataFrame(data).transpose()
    data.columns = ['date','beta','gamma','r0', 'dif','r0_p','dif_p']
    data.to_csv(dir_path +'paramPredict.csv')

def parameterPlot(beta, gamma):
    beta = beta.tolist()
    gamma = gamma.tolist()
    r =[]
    ex = []
    for i in range(0, len(beta)-1):
        r.append(beta[i]/gamma[i])
        ex.append(beta[i]-gamma[i])

    return r, ex

def predictParameter(data_x, data_y):
    x_train, x_test, y_train, y_test = train_test_split(data_x[:-50], data_y, shuffle=False)
    quadratic = PolynomialFeatures(degree=2)
    x_train_quad = quadratic.fit_transform(x_train)
    x_test_quad = quadratic.transform(x_test)

    regressor = linear_model.LinearRegression()
    regressor.fit(x_train_quad, y_train)
    xx_quad = quadratic.transform(data_x.reshape(data_x.shape[0], 1))
    print('r-squared', regressor.score(x_test_quad, y_test))
    return regressor.predict(xx_quad)

def predictSomething(country, size, t):
    I,R,D,C = load_IR(country,popu_dict[country],0)
    beta_list,gamma_list = read_opt(I,country, size)
    r0,dif = parameterPlot(beta_list[:t],gamma_list[:t])
    leng = len(r0)+50
    datasets_X = np.arange(0, leng).reshape([leng,1])
    dif_new = predictParameter(datasets_X, dif)
    r0_new = predictParameter(datasets_X, r0)
    plt.figure()
    plt.plot(datasets_X[:len(dif)],dif, label="Ex(t) aus Modell", linewidth=0.5)
    plt.plot(datasets_X, dif_new, label='Ex(t) predicted')
    plt.title('Ex(t)')
    plt.grid(True)
    plt.figure()
    plt.title('R0(t)')
    plt.plot(datasets_X[:len(r0)], r0, linewidth=0.5)
    plt.plot(datasets_X, r0_new)
    plt.grid(True)
    gamma =dif_new/(r0_new-1)
    beta = r0_new*gamma
    I_t_p, I_t_all = read_I_t(I, country, size)
    I_p = list(I_t_p)[:t]
    for i in range(0,50):
        S_t, I_t, R_t = RungeKutta(beta[len(r0)+i], gamma[len(r0)+i], popu_dict[country],
                                   popu_dict[country] - I[len(r0)+i] - R[len(r0)+i], I[len(r0)+i], R[len(r0)+i], 0)
        I_p.append(I_t[size])

    I_p = np.array(I_p)
    predictPlot(I, I_p, t)

In [5]:
def predictTime(country, size, t):
    I,R,D,C = load_IR(country,popu_dict[country],0)
    beta_list,gamma_list = read_opt(I,country, size)
    print(len(beta_list))
    r0,dif = parameterPlot(beta_list[:t],gamma_list[:t])
    #r0hat = scipy.signal.savgol_filter(r0, 15, 5)
    #difhat = scipy.signal.savgol_filter(dif, 15, 5)
    datasets_X = np.arange(0, len(r0)).reshape([len(r0),1])
    dif_new = paramPredict(datasets_X[-size:], dif[-size:])
    r0_new = paramPredict(datasets_X[-size:], r0[-size:])
    gamma = dif_new / (r0_new - 1)
    beta = r0_new * gamma
    #print(beta, gamma)
    I_t_p,I_t_all = read_I_t(I,country,size)
    print('Length I_t', len(I_t_p))
    S_t,I_t,R_t = RungeKutta(beta, gamma, popu_dict[country], popu_dict[country] - I[t-1] - R[t-1], I[t-1], R[t-1], 0)
    I_p = list(I_t_p)[:t]
    I_p = I_p + list(I_t)
    #I_p.append(I_t)
    I_p = np.array(I_p)
    predictPlot(I[0:t+80],I_p[0:t+80], t)
    dirs = '\\data\\'
    dir_path = os.path.dirname(os.path.abspath('__file__')) + dirs
    dir_path = dir_path+country+'\\'+'Picture'+'\\'
    plt.savefig(dir_path + 'future%i.png' %t)

    C_p = list(C)
    ax  = plt.figure(figsize=(13, 8))
    for i in range(len(C), len(I_p)):
        a = C_p[i-1] + (C_p[i-1] - C_p[i-2])*0.97
        C_p.append(a)

    dirs = '\\data\\'
    dir_path = os.path.dirname(os.path.abspath('__file__')) + dirs
    dir_path = dir_path+country+'\\'
    date_X = create_assist_date("2020-1-22", "2021-10-01")
    data = []
    for i in range(0, t+50):
        if i < len(I):
            data.append((date_X[i], I[i],R[i],D[i],C[i],I_p[i],C_p[i]))
        else:
            data.append((date_X[i], '', '', '', '', I_p[i],C_p[i]))
    df = pd.DataFrame(data)
    df.columns=['date','I','R', 'D', 'C', 'I_p','C_p']
    df.to_csv(dir_path +'predictTime.csv')
    return I, I_p

In [6]:
def justPredict(size, country, t, r0, dif, datasets_X, I, R,  forecastType, smoothday=None):

    if forecastType == 'normal':
        dif_new = paramPredict(datasets_X[-size:], dif[-size:])
        r0_new = paramPredict(datasets_X[-size:], r0[-size:])
    elif forecastType == 'smooth15':
        r0hat = scipy.signal.savgol_filter(r0, smoothday, 2)
        difhat = scipy.signal.savgol_filter(dif, smoothday, 2)
        dif_new = paramPredict(datasets_X[-size:], difhat[-size:])
        r0_new = paramPredict(datasets_X[-size:], r0hat[-size:])
    elif forecastType == 'more':
        dif_new = paramPredict(datasets_X[-31:], dif[-31:])
        r0_new = paramPredict(datasets_X[-31:], r0[-31:])
    elif forecastType == 'allDays':
        dif_new = paramPredict(datasets_X, dif)
        r0_new = paramPredict(datasets_X, r0)
    elif forecastType == 'mix':
        r0hat = scipy.signal.savgol_filter(r0, 15, 2)
        difhat = scipy.signal.savgol_filter(dif, 15, 2)
        dif_new = paramPredict(datasets_X[-31:], difhat[-31:])
        r0_new = paramPredict(datasets_X[-31:], r0hat[-31:])

    gamma = dif_new / (r0_new - 1)
    beta = r0_new * gamma
    I_t_p,I_t_all = read_I_t(I,country,size)
    S_t,I_t,R_t = RungeKutta(beta, gamma, popu_dict[country], popu_dict[country] - I[t-1] - R[t-1], I[t-1], R[t-1], 0)
    I_p = list(I_t_p)[:t]
    I_p = I_p + list(I_t)
    I_p = np.array(I_p)
    return I_p

def testForecast(testrange):
    """
    Testet verschiedene Arten von Vorhersagen
    :param testrange: ab wann wird vorhergesagt
    :return: Matrix mit allen Fehlern
    """
    country = 'Germany'
    size = 5
    forecastType = ['normal', 'smooth15', 'more', 'allDays', 'mix']
    # normal: normale Vorhersage
    # smooth: geglaettete Basisreproduktionszahl und exp. Wachstumsrate
    # more: pol. Regresion mit den letzten 31 Tagen
    # allDays: pol. Regression mit allen Tagen
    # mix: pol. Regression mit letzten 31 Tagen und geglaettet
    smoothdays = [7, 15, 31, 63] # wie viele Tage werden geglaettet
    I,R,D,C = load_IR(country,popu_dict[country],0)
    beta_list,gamma_list = read_opt(I,country, size)
    errorMatrix = []
    for t in testrange:
        tempError1 = [t, 1] # error fuer ersten Tag
        tempError2 = [t, 2] # error fuer zweiten Tag
        tempError3 = [t, 3] # error fuer driiten Tag
        r0,dif = parameterPlot(beta_list[:t],gamma_list[:t])
        datasets_X = np.arange(0, len(r0)).reshape([len(r0),1])
        for i in forecastType:
            if i == 'smooth15':
                for j in smoothdays:
                    err = 0.0
                    err2 = 0.0
                    err3 = 0.0
                    if j < t:
                        I_p = justPredict(size, country,t, r0, dif, datasets_X,I,R ,i, j)
                        err = (I[t + 1] - I_p[t + 1]) / I[t + 1]
                        err2 = (I[t + 2] - I_p[t + 2]) / I[t + 2]
                        err3 = (I[t + 3] - I_p[t + 3]) / I[t + 3]
                    tempError1.append(err)
                    tempError2.append(err2)
                    tempError3.append(err3)

            else:
                I_p = justPredict(size, country,t, r0, dif, datasets_X,I,R, i)
                err = (I[t + 1] - I_p[t + 1]) / I[t + 1]
                err2 = (I[t + 2] - I_p[t + 2]) / I[t + 2]
                err3 = (I[t + 3] - I_p[t + 3]) / I[t + 3]
                tempError1.append(err)
                tempError2.append(err2)
                tempError3.append(err3)
        errorMatrix.append(tempError1)
        errorMatrix.append(tempError2)
        errorMatrix.append(tempError3)

    return errorMatrix

In [7]:
# Auswertungen

country = 'Germany'
# testrange = [106, 290, 337, 459]
# error = testForecast(testrange)
# print(error[0])
# print(error[1])
# print([i[2] for i in error])
#I, I_p = predictTime(country, 5, 106)

In [8]:
country = 'Germany'
forecastType = ['normal', 'smooth15', 'more', 'allDays', 'mix']
smoothdays = [7, 15, 31] # wie viele Tage werden geglaettet
I,R,D,C = load_IR(country,popu_dict[country],0)
beta_list,gamma_list = read_opt(I,country, 5)
error1 =[]
error2=[]
error3=[]
for t in range(70,400):
    r0,dif = parameterPlot(beta_list[:t],gamma_list[:t])
    datasets_X = np.arange(0, len(r0)).reshape([len(r0),1])
    for n in smoothdays:
        I_p = justPredict(5, country, t, r0, dif, datasets_X, I, R,  'smooth15', n)
        error1.append(abs(I[t + 1] - I_p[t + 1]) / I[t + 1])
        error2.append(abs(I[t + 2] - I_p[t + 2]) / I[t + 2])
        error3.append(abs(I[t + 3] - I_p[t + 3]) / I[t + 3])


<ipython-input-3-cb85ba15e97a>:71: RuntimeWarning: overflow encountered in multiply
  k14 = -beta * (S_t[i-1] + h * k13) * (I_t[i-1] + h * k23) / N
<ipython-input-3-cb85ba15e97a>:72: RuntimeWarning: overflow encountered in multiply
  k24 = beta * (S_t[i-1] + h * k13) * (I_t[i-1] + h * k23) / N - gamma * (I_t[i-1] + h * k23)
<ipython-input-3-cb85ba15e97a>:59: RuntimeWarning: overflow encountered in multiply
  k11 = -beta * S_t[i-1] * I_t[i-1] / N
<ipython-input-3-cb85ba15e97a>:60: RuntimeWarning: overflow encountered in multiply
  k21 = beta * S_t[i-1] * I_t[i-1] / N - gamma * I_t[i-1]
<ipython-input-3-cb85ba15e97a>:71: RuntimeWarning: overflow encountered in multiply
  k14 = -beta * (S_t[i-1] + h * k13) * (I_t[i-1] + h * k23) / N
<ipython-input-3-cb85ba15e97a>:72: RuntimeWarning: overflow encountered in multiply
  k24 = beta * (S_t[i-1] + h * k13) * (I_t[i-1] + h * k23) / N - gamma * (I_t[i-1] + h * k23)
<ipython-input-3-cb85ba15e97a>:71: RuntimeWarning: overflow encountered in multipl

In [20]:
print(np.mean(error1))
print(np.mean(error2))
print(np.mean(error3))
test = error3[0:len(error2):3]
print(np.mean(test))
test2 = error3[1:len(error2):3]
test3 = error3[2:len(error2):3]
print(np.mean(test2))
print(np.mean(test3))

0.04238316400750818
0.05987051602699927
0.07985959982419615
0.09817749417560974
0.07199980433366936
0.06940150096330938


In [10]:
error11 =[]
error21=[]
error31=[]
for t in range(70,400):
    r0,dif = parameterPlot(beta_list[:t],gamma_list[:t])
    datasets_X = np.arange(0, len(r0)).reshape([len(r0),1])
    I_p = justPredict(5, country, t, r0, dif, datasets_X, I, R,  'normal')
    error11.append(abs(I[t + 1] - I_p[t + 1]) / I[t + 1])
    error21.append(abs(I[t + 2] - I_p[t + 2]) / I[t + 2])
    error31.append(abs(I[t + 3] - I_p[t + 3]) / I[t + 3])

<ipython-input-3-cb85ba15e97a>:67: RuntimeWarning: overflow encountered in multiply
  k13 = -beta * (S_t[i-1] + h / 2 * k12) * (I_t[i-1] + h / 2 * k22) / N
<ipython-input-3-cb85ba15e97a>:68: RuntimeWarning: overflow encountered in multiply
  k23 = beta * (S_t[i-1] + h / 2 * k12) * (I_t[i-1] + h / 2 * k22) / N - gamma * (I_t[i-1] + h / 2 * k22)
<ipython-input-3-cb85ba15e97a>:71: RuntimeWarning: overflow encountered in multiply
  k14 = -beta * (S_t[i-1] + h * k13) * (I_t[i-1] + h * k23) / N
<ipython-input-3-cb85ba15e97a>:72: RuntimeWarning: overflow encountered in multiply
  k24 = beta * (S_t[i-1] + h * k13) * (I_t[i-1] + h * k23) / N - gamma * (I_t[i-1] + h * k23)


In [11]:
print(np.mean(error11))
print(np.mean(error21))
print(np.mean(error31))

#Test welches Fenster bei Vorhersage am besten
country = 'Germany'
# end = [106, 290, 337, 459]
# for i in end:
#     I, I_p = predictTime(country, 5, i)
#     error = (I[i + 1] - I_p[i + 1]) / I[i + 1]
#     error2 = (I[i + 2] - I_p[i + 2]) / I[i + 2]
#     error3 = (I[i + 3] - I_p[i + 3]) / I[i + 3]
# #     #er = oneDayError(I[end[i]:end[i]+3], I_p[end[i]:end[i]+3])
#     print("predictTime: \nsize:", i, "Fehler 1:", error)
#     print("size:", i, "Fehler 2:", error2)
#     print("size:", i, "Fehler 3:", error3)
#predictSomething(country, 5, 290)

0.040562057225062134
0.06567956638609426
0.0963192111456
